In [1]:
import pandas as pd
import numpy as np

ACADEMIC_PATH = r'../../data/raw/academic_records.csv'
ADMISSION_PATH = r'../../data/raw/admission.csv'
TEST_PATH = r'../../data/raw/test.csv'
academic_records = pd.read_csv(ACADEMIC_PATH)
admission = pd.read_csv(ADMISSION_PATH)

In [2]:
import pandas as pd
import numpy as np
import re

# --- HÀM MỚI: Xử lý định dạng học kỳ phức tạp ---
def parse_semester_string(sem_str):
    """
    Chuyển đổi chuỗi như 'HK1 2023-2024' thành mã số 20231 để sort được.
    Logic: Năm * 10 + Kỳ
    """
    s = str(sem_str).strip()
    
    # Trường hợp 1: Dạng số sẵn (VD: 20231)
    if s.isdigit():
        return int(s)
    
    # Trường hợp 2: Dạng chữ (VD: HK1 2023-2024 hoặc Học kỳ 1 năm 2023)
    # Tìm tất cả các con số trong chuỗi
    digits = re.findall(r'\d+', s)
    
    if len(digits) >= 2:
        # Giả sử số nhỏ là kỳ, số lớn (4 chữ số) là năm
        # Tìm năm (thường là số có 4 chữ số đầu tiên tìm thấy)
        years = [int(d) for d in digits if len(d) == 4]
        sems = [int(d) for d in digits if len(d) == 1]
        
        if years and sems:
            year = years[0]
            sem = sems[0]
            return year * 10 + sem
            
    return 0 # Không xác định

def clean_data_pipeline_v3(admission, academic_records):
    print("--- 🚀 BẮT ĐẦU QUY TRÌNH LÀM SẠCH DỮ LIỆU (FIXED VERSION) ---")
    
    adm = admission.copy()
    acad = academic_records.copy()
    
    # 1. Chuẩn hóa ID
    adm['MA_SO_SV'] = adm['MA_SO_SV'].astype(str)
    acad['MA_SO_SV'] = acad['MA_SO_SV'].astype(str)
    
    # ---------------------------------------------------------
    # [FIX QUAN TRỌNG] Xử lý HOC_KY thông minh hơn
    # ---------------------------------------------------------
    print("-> Đang xử lý cột HOC_KY...")
    # Tạo cột HOC_KY_INT dùng để sort (VD: 20231)
    acad['HOC_KY_INT'] = acad['HOC_KY'].apply(parse_semester_string)
    
    # Kiểm tra xem có dòng nào bị lỗi (bằng 0) không
    error_count = (acad['HOC_KY_INT'] == 0).sum()
    if error_count > 0:
        print(f"   ⚠️ Cảnh báo: Có {error_count} dòng không đọc được HOC_KY.")

    # Merge dữ liệu
    df = pd.merge(acad, adm, on='MA_SO_SV', how='left')
    
    # Sắp xếp theo Time-series chuẩn xác dựa trên cột vừa tạo
    df = df.sort_values(by=['MA_SO_SV', 'HOC_KY_INT']).reset_index(drop=True)
    
    # ---------------------------------------------------------
    # 2. XỬ LÝ SỐ LIỆU & LOGIC
    # ---------------------------------------------------------
    cols_float = ['GPA', 'CPA', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN']
    cols_int = ['TC_DANGKY', 'TC_HOANTHANH']
    
    for col in cols_float:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
    for col in cols_int:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Logic: Hoàn thành <= Đăng ký
    df['TC_HOANTHANH'] = np.minimum(df['TC_HOANTHANH'], df['TC_DANGKY'])
    
    # Target Transformation
    df['COMPLETION_RATE'] = df['TC_HOANTHANH'] / (df['TC_DANGKY'] + 1e-9)
    df['COMPLETION_RATE'] = df['COMPLETION_RATE'].clip(0, 1)

    # Clip điểm số
    df['GPA'] = df['GPA'].clip(0, 4.0)
    df['CPA'] = df['CPA'].clip(0, 4.0)

    # Admission Gap Feature
    if 'DIEM_TRUNGTUYEN' in df.columns and 'DIEM_CHUAN' in df.columns:
        df['ADMISSION_GAP'] = df['DIEM_TRUNGTUYEN'] - df['DIEM_CHUAN']
    
    # Lọc rác
    initial_len = len(df)
    df = df[df['TC_DANGKY'] > 0]
    
    print(f"--- ✅ HOÀN TẤT. Kích thước data: {df.shape} ---")
    print("Sample HOC_KY_INT:", df['HOC_KY_INT'].head().tolist())
    
    return df

# Chạy thử lại
df_clean = clean_data_pipeline_v3(admission, academic_records)
df_clean.head()

--- 🚀 BẮT ĐẦU QUY TRÌNH LÀM SẠCH DỮ LIỆU (FIXED VERSION) ---
-> Đang xử lý cột HOC_KY...
--- ✅ HOÀN TẤT. Kích thước data: (105726, 14) ---
Sample HOC_KY_INT: [20231, 20232, 20211, 20212, 20221]


,MA_SO_SV,HOC_KY,CPA,GPA,TC_DANGKY,TC_HOANTHANH,HOC_KY_INT,NAM_TUYENSINH,PTXT,TOHOP_XT,DIEM_TRUNGTUYEN,DIEM_CHUAN,COMPLETION_RATE,ADMISSION_GAP
0,00003e092652,HK1 2023-2024,1.64,1.97,18,15,20231,2023,100,A00,21.32,20.25,0.833333,1.07
1,00003e092652,HK2 2023-2024,1.53,2.05,18,13,20232,2023,100,A00,21.32,20.25,0.722222,1.07
2,000e15519006,HK1 2021-2022,3.85,3.85,9,9,20211,2021,1,D07,23.84,22.43,1.000000,1.41
3,000e15519006,HK2 2021-2022,2.77,3.12,19,19,20212,2021,1,D07,23.84,22.43,1.000000,1.41
4,000e15519006,HK1 2022-2023,2.83,2.98,21,21,20221,2021,1,D07,23.84,22.43,1.000000,1.41


In [3]:
import pandas as pd
import numpy as np
import re

# ==============================================================================
# 1. CÁC HÀM TIỆN ÍCH (UTILS)
# ==============================================================================

def parse_semester_string(sem_str):
    """
    Chuyển đổi 'HK1 2023-2024' -> 20231 (Int) để sort time-series.
    """
    s = str(sem_str).strip()
    if s.isdigit(): return int(s)
    
    digits = re.findall(r'\d+', s)
    if len(digits) >= 2:
        years = [int(d) for d in digits if len(d) == 4]
        sems = [int(d) for d in digits if len(d) == 1]
        if years and sems:
            return years[0] * 10 + sems[0]
    return 0

def fast_slope(y):
    """
    Tính hệ số góc (Trend) nhanh.
    Input: Array numpy (có thể chứa NaN).
    """
    # Lọc bỏ NaN trước khi tính (Quan trọng!)
    y_clean = y[~np.isnan(y)]
    n = len(y_clean)
    if n < 2: return 0.0
    
    x = np.arange(n)
    x_mean = np.mean(x)
    y_mean = np.mean(y_clean)
    
    numerator = np.sum((x - x_mean) * (y_clean - y_mean))
    denominator = np.sum((x - x_mean) ** 2)
    
    return numerator / (denominator + 1e-6)

# ==============================================================================
# 2. DATA PREPROCESSING PIPELINE
# ==============================================================================

def clean_data_pipeline(admission, academic_records, is_test=False):
    print("--- 🚀 1. PREPROCESSING DATA ---")
    adm = admission.copy()
    acad = academic_records.copy()
    
    # Chuẩn hóa ID
    adm['MA_SO_SV'] = adm['MA_SO_SV'].astype(str)
    acad['MA_SO_SV'] = acad['MA_SO_SV'].astype(str)
    
    # Tạo Time-Index
    acad['semester_order'] = acad['HOC_KY'].apply(parse_semester_string)
    
    # Merge
    df = pd.merge(acad, adm, on='MA_SO_SV', how='left')
    
    # Sort Time-Series (CỰC KỲ QUAN TRỌNG)
    df = df.sort_values(by=['MA_SO_SV', 'semester_order']).reset_index(drop=True)
    
    # Numeric conversion
    cols_float = ['GPA', 'CPA', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN']
    cols_int = ['TC_DANGKY', 'TC_HOANTHANH']
    
    for col in cols_float:
        if col in df.columns: df[col] = pd.to_numeric(df[col], errors='coerce')
    for col in cols_int:
        if col in df.columns: df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Logic clean
    if not is_test:
        df['TC_HOANTHANH'] = np.minimum(df['TC_HOANTHANH'], df['TC_DANGKY'])
        df['GPA'] = df['GPA'].clip(0, 4.0)
        df['CPA'] = df['CPA'].clip(0, 4.0)
    
    # Target Transformation (Chỉ dùng cho train)
    if not is_test:
        df['COMPLETION_RATE'] = df['TC_HOANTHANH'] / (df['TC_DANGKY'] + 1e-9)
        df['COMPLETION_RATE'] = df['COMPLETION_RATE'].clip(0, 1)

    return df

# ==============================================================================
# 3. FEATURE ENGINEERING CLASS (CORE)
# ==============================================================================

class FeatureEngineer:    
    def __init__(self):
        # Các cột Category sẽ giữ lại
        self.cat_cols = ['PTXT', 'TOHOP_XT', 'MA_NGANH', 'KV_UT', 'KHOA_VIEN'] 

    def create_features(self, df):
        print("--- ⚡ 2. FEATURE ENGINEERING (FULL POWER) ---")
        df = df.copy()
        
        # Sort lại cho chắc chắn
        df = df.sort_values(['MA_SO_SV', 'semester_order']).reset_index(drop=True)
        
        # Convert Category
        for col in self.cat_cols:
            if col in df.columns:
                df[col] = df[col].astype(str).astype('category')

        # Groupby object
        g = df.groupby('MA_SO_SV')

        # ---------------------------------------------------------
        # A. BASE LAGS (Cực kỳ quan trọng: Dùng shift(1) để tránh Leakage)
        # ---------------------------------------------------------
        # Tạo biến Raw (chứa NaN) để tính toán thống kê chính xác hơn
        df['Prev_GPA_Raw'] = g['GPA'].shift(1)
        
        # Tạo biến Fill (để model dùng trực tiếp)
        df['Prev_GPA'] = df['Prev_GPA_Raw'].fillna(-1)
        df['Prev_CPA'] = g['CPA'].shift(1).fillna(-1)
        df['Prev_TC_HOANTHANH'] = g['TC_HOANTHANH'].shift(1).fillna(0)
        df['Prev_TC_DANGKY'] = g['TC_DANGKY'].shift(1).fillna(0)
        
        # Cờ năm nhất (Chưa có lịch sử)
        df['is_freshman'] = (df['Prev_TC_DANGKY'] == 0).astype(int)

        # ---------------------------------------------------------
        # B. GỌI CÁC NHÓM FEATURE
        # ---------------------------------------------------------
        df = self._create_admission_features(df)
        df = self._create_history_features(df)
        df = self._create_trend_features(df)  # <--- Slope, Volatility nằm ở đây
        df = self._create_risk_features(df)
        
        # Dọn dẹp cột tạm
        if 'Prev_GPA_Raw' in df.columns:
            df = df.drop(columns=['Prev_GPA_Raw'])
            
        return df

    def _create_admission_features(self, df):
        # Khoảng cách điểm đầu vào
        if 'DIEM_TRUNGTUYEN' in df.columns and 'DIEM_CHUAN' in df.columns:
            df['diem_vuot_chuan'] = df['DIEM_TRUNGTUYEN'] - df['DIEM_CHUAN']
        
        # Tuổi (giả định năm hiện tại 2025/2026)
        if 'NAM_TUYENSINH' in df.columns:
            df['nam_tuoi'] = 2026 - df['NAM_TUYENSINH']
            
        # Số kỳ đã học (tính từ năm nhập học)
        df['semester_number'] = df.groupby('MA_SO_SV').cumcount() + 1
        return df

    def _create_history_features(self, df):
        # 1. Delta GPA-CPA: Đang học tốt hơn hay tệ hơn trung bình tích lũy?
        df['prev_gpa_cpa_diff'] = df['Prev_GPA'] - df['Prev_CPA']
        
        # 2. Tỷ lệ hoàn thành kỳ trước
        df['prev_completion_rate'] = df['Prev_TC_HOANTHANH'] / (df['Prev_TC_DANGKY'] + 1e-9)
        
        # 3. Load Factor (Áp lực): Đăng ký kỳ này / Sức học trung bình
        avg_capacity = df.groupby('MA_SO_SV')['Prev_TC_HOANTHANH'].transform(
            lambda x: x.rolling(window=5, min_periods=1).mean()
        ).fillna(15)
        
        df['load_factor'] = df['TC_DANGKY'] / (avg_capacity + 1e-9)
        
        # Biến cờ: Rớt môn kỳ trước
        df['failed_last_sem'] = (df['Prev_TC_HOANTHANH'] < df['Prev_TC_DANGKY']).astype(int)
        
        return df

    def _create_trend_features(self, df):
        # Sử dụng Prev_GPA_Raw (có NaN) để tính Slope chính xác
        # Nếu dùng Prev_GPA (có -1), Slope sẽ bị sai lệch lớn
        g_raw = df.groupby('MA_SO_SV')['Prev_GPA_Raw']
        
        # 1. GPA Slope (Xu hướng điểm)
        df['gpa_trend_slope'] = g_raw.transform(
            lambda x: x.rolling(window=3, min_periods=2).apply(fast_slope, raw=True)
        ).fillna(0)
        
        # 2. [RECOVERED] GPA Volatility (Độ ổn định)
        df['gpa_volatility'] = g_raw.transform(
            lambda x: x.rolling(window=4, min_periods=2).std()
        ).fillna(0)
        
        # 3. Tích lũy tín chỉ (History)
        grouped = df.groupby('MA_SO_SV')
        cum_dangky = grouped['Prev_TC_DANGKY'].cumsum()
        cum_hoanthanh = grouped['Prev_TC_HOANTHANH'].cumsum()
        
        df['total_credits_failed'] = cum_dangky - cum_hoanthanh
        df['accumulated_fail_ratio'] = df['total_credits_failed'] / (cum_dangky + 1e-9)
        
        # 4. [RECOVERED] Credit Velocity (Tốc độ học)
        semester_count = grouped.cumcount() + 1
        df['credit_velocity'] = cum_hoanthanh / semester_count
        
        return df

    def _create_risk_features(self, df):
        # Hành vi "Gỡ gạc" (Aggressive Recovery): Rớt môn kỳ trước -> Đăng ký nhiều hơn kỳ này
        more_credits = (df['TC_DANGKY'] > df['Prev_TC_DANGKY'])
        df['aggressive_recovery'] = (df['failed_last_sem'] & more_credits).astype(int)
        
        # Kỳ vọng hoàn thành (Expected Credits) = Đăng ký * Tỷ lệ đậu toàn cục của SV đó
        df['expected_real_credits'] = df['TC_DANGKY'] * (1 - df['accumulated_fail_ratio'])
        
        return df

    def get_feature_columns(self, df):
        """Tự động lấy danh sách feature dựa trên prefix"""
        
        # Whitelist các prefix
        valid_prefixes = [
            'Prev_', 'prev_', 'sem_', 'diem_', 'nam_', 'is_', 
            'load_', 'aggressive_', 'gpa_', 'total_', 'accumulated_',
            'credit_', 'expected_', 'failed_'
        ]
        
        valid_exact = ['TC_DANGKY', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN', 'semester_number']
        valid_exact.extend(self.cat_cols)
        
        final_cols = []
        # Các cột target/meta cần loại bỏ
        ignore_cols = ['TC_HOANTHANH', 'GPA', 'CPA', 'semester_order', 'MA_SO_SV', 'HOC_KY', 'COMPLETION_RATE', 'Prev_GPA_Raw']
        
        for col in df.columns:
            if col in ignore_cols: continue
            
            is_valid = False
            if col in valid_exact: is_valid = True
            else:
                for prefix in valid_prefixes:
                    if col.startswith(prefix):
                        is_valid = True
                        break
            
            if is_valid: final_cols.append(col)
                
        return final_cols

# ==============================================================================
# 4. EXECUTION BLOCK (CHẠY THỬ)
# ==============================================================================


test_raw = pd.read_csv(TEST_PATH)

print("--- BẮT ĐẦU PIPELINE ---")

# 1. Clean Data
df_train_raw = clean_data_pipeline(admission, academic_records, is_test=False)

# 2. Chuẩn bị Test (Gán dummy values để chạy được pipeline)
test_copy = test_raw.copy()
for col in ['TC_HOANTHANH', 'GPA', 'CPA']: test_copy[col] = 0
df_test_raw = clean_data_pipeline(admission, test_copy, is_test=True)

# 3. Gộp Train + Test để tạo feature (Tránh biên giới giữa các kỳ bị đứt gãy)
df_train_raw['set_type'] = 'TRAIN'
df_test_raw['set_type'] = 'TEST'
full_df = pd.concat([df_train_raw, df_test_raw], ignore_index=True)

# 4. Feature Engineering
fe = FeatureEngineer()
full_df_fe = fe.create_features(full_df)

# 5. Tách lại Train/Test
train_final = full_df_fe[full_df_fe['set_type'] == 'TRAIN'].copy()
test_final = full_df_fe[full_df_fe['set_type'] == 'TEST'].copy()

# 6. Lấy features & Target
feature_cols = fe.get_feature_columns(train_final)
X = train_final[feature_cols]
y = train_final['TC_HOANTHANH'] # Hoặc dùng COMPLETION_RATE nếu muốn

print(f"✅ Xong! Số lượng Features: {len(feature_cols)}")
print(f"Sample Features: {feature_cols[:5]}")

--- BẮT ĐẦU PIPELINE ---
--- 🚀 1. PREPROCESSING DATA ---
--- 🚀 1. PREPROCESSING DATA ---
--- ⚡ 2. FEATURE ENGINEERING (FULL POWER) ---
✅ Xong! Số lượng Features: 24
Sample Features: ['TC_DANGKY', 'PTXT', 'TOHOP_XT', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN']


In [4]:
# ==============================================================================
# 5. CHUẨN BỊ DỮ LIỆU CHO TRAINING (THE BRIDGE)
# ==============================================================================
print("\n--- ✂️ 3. SPLITTING TRAIN/VALID/TEST ---")

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Định nghĩa mốc thời gian (Theo đề bài)
SPLIT_SEM = 20231  # Train đến hết HK1 2023-2024
VALID_SEM = 20232  # Valid là HK2 2023-2024

# Tạo Mask để lọc dữ liệu
# Lưu ý: train_final của bạn đang chứa cả Train và Valid
mask_train = train_final['semester_order'] <= SPLIT_SEM
mask_valid = train_final['semester_order'] == VALID_SEM

# 1. Định nghĩa tập TRAIN (Dùng để dạy model)
X_train = train_final[mask_train][feature_cols]
# Model học Tỷ lệ (Rate) chứ không học số tín chỉ -> Quy về [0, 1]
y_train = train_final[mask_train]['TC_HOANTHANH'] / (train_final[mask_train]['TC_DANGKY'] + 1e-9)
y_train = y_train.clip(0, 1)

# 2. Định nghĩa tập VALID (Dùng để kiểm tra & Stacking)
X_valid = train_final[mask_valid][feature_cols]
y_valid_rate = train_final[mask_valid]['TC_HOANTHANH'] / (train_final[mask_valid]['TC_DANGKY'] + 1e-9)
y_valid_rate = y_valid_rate.clip(0, 1)
# Các biến dùng để đo lường RMSE thực tế
y_valid_credits = train_final[mask_valid]['TC_HOANTHANH'] 
valid_dangky = train_final[mask_valid]['TC_DANGKY']

# 3. Định nghĩa tập TEST (Dùng để nộp bài)
X_test = test_final[feature_cols]
test_dangky = test_final['TC_DANGKY']

# Xác định danh sách cột Category cho CatBoost
# (Lấy từ feature_cols những cột nằm trong danh sách cat_cols của bạn)
cat_features_list = ['PTXT', 'TOHOP_XT', 'MA_NGANH', 'KV_UT', 'KHOA_VIEN']
valid_cat_features = [c for c in feature_cols if c in cat_features_list]

print(f"📌 Train size: {X_train.shape}")
print(f"📌 Valid size: {X_valid.shape}")
print(f"📌 Test size:  {X_test.shape}")
print(f"📌 Categorical Features: {valid_cat_features}")

# ==============================================================================
# 6. HUẤN LUYỆN STACKING ENSEMBLE (LAYER 1)
# ==============================================================================
print("\n--- 🚀 4. ACTIVATING STACKING ENSEMBLE ---")

# --- MODEL 1: XGBOOST ---
print("   -> Training XGBoost...")
xgb_params = {
    'n_estimators': 2000,
    'learning_rate': 0.02,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'reg:squarederror',
    'n_jobs': -1,
    'random_state': 42,
    'enable_categorical': True, # Đã bật hỗ trợ category
    'tree_method': 'hist',
    'early_stopping_rounds': 100
}
model_xgb = xgb.XGBRegressor(**xgb_params)
model_xgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid_rate)], verbose=False)

# --- MODEL 2: LIGHTGBM ---
print("   -> Training LightGBM...")
lgb_params = {
    'n_estimators': 2000,
    'learning_rate': 0.02,
    'num_leaves': 31,
    'objective': 'rmse',
    'metric': 'rmse',
    'n_jobs': -1,
    'random_state': 42,
    'verbose': -1
}
model_lgb = lgb.LGBMRegressor(**lgb_params)
model_lgb.fit(
    X_train, y_train, 
    eval_set=[(X_valid, y_valid_rate)], 
    eval_metric='rmse',
    callbacks=[lgb.early_stopping(100), lgb.log_evaluation(0)]
)

# --- MODEL 3: CATBOOST ---
print("   -> Training CatBoost...")
cat_params = {
    'iterations': 2000,
    'learning_rate': 0.02,
    'depth': 6,
    'loss_function': 'RMSE',
    'verbose': 0, # Tắt log cho gọn
    'random_state': 42,
    'allow_writing_files': False,
    'cat_features': valid_cat_features 
}
model_cat = CatBoostRegressor(**cat_params)
model_cat.fit(X_train, y_train, eval_set=(X_valid, y_valid_rate), verbose=False)

# ==============================================================================
# 7. LAYER 2: META-LEARNER (RIDGE REGRESSION) & PREDICTION
# ==============================================================================
print("\n--- 🔄 Layer 2: Blending & Prediction ---")

# 1. Tạo Meta-Features (Dự báo của 3 model trên tập Valid và Test)
def get_preds(model, X):
    return np.clip(model.predict(X), 0, 1)

# Predict Valid
pred_xgb_valid = get_preds(model_xgb, X_valid)
pred_lgb_valid = get_preds(model_lgb, X_valid)
pred_cat_valid = get_preds(model_cat, X_valid)

# Predict Test
pred_xgb_test = get_preds(model_xgb, X_test)
pred_lgb_test = get_preds(model_lgb, X_test)
pred_cat_test = get_preds(model_cat, X_test)

# 2. Xây dựng Dataset cho Layer 2
X_meta_valid = pd.DataFrame({'XGB': pred_xgb_valid, 'LGB': pred_lgb_valid, 'CAT': pred_cat_valid})
X_meta_test = pd.DataFrame({'XGB': pred_xgb_test, 'LGB': pred_lgb_test, 'CAT': pred_cat_test})

# 3. Train Meta-Learner (Ridge)
meta_model = Ridge(alpha=10.0)
meta_model.fit(X_meta_valid, y_valid_rate)

# Xem trọng số
weights = meta_model.coef_
print(f"📊 Trọng số Stacking: XGB: {weights[0]:.2f} | LGB: {weights[1]:.2f} | CAT: {weights[2]:.2f}")

# 4. Dự báo cuối cùng (Rate)
final_rate_valid = meta_model.predict(X_meta_valid)
final_rate_test = meta_model.predict(X_meta_test)

# ==============================================================================
# 8. HẬU XỬ LÝ & TẠO FILE NỘP (SUBMISSION)
# ==============================================================================

def post_process_credits(pred_rate, registered_credits):
    """Chuyển Rate -> Credits với các ràng buộc vật lý"""
    # Clip Rate
    pred_rate = np.clip(pred_rate, 0, 1)
    
    # Magic Trick: Nếu rất tự tin (>96%), đẩy lên 100%
    pred_rate[pred_rate >= 0.96] = 1.0
    
    # Convert ra tín chỉ
    pred_credits = pred_rate * registered_credits
    
    # Hard Limit: Không vượt quá đăng ký
    pred_credits = np.minimum(pred_credits, registered_credits)
    
    return pred_credits

# Đánh giá RMSE trên tập Valid
final_credits_valid = post_process_credits(final_rate_valid, valid_dangky)
rmse_stacking = np.sqrt(mean_squared_error(y_valid_credits, final_credits_valid))
print(f"\n>>> ✅ RMSE Stacking (Valid - HK2 23/24): {rmse_stacking:.4f}")

# Tạo file nộp bài cho Test (HK1 24/25)
final_credits_test = post_process_credits(final_rate_test, test_dangky)
final_submission = final_credits_test


submission_df = pd.DataFrame({
    'MA_SO_SV': test_final['MA_SO_SV'],
    'PRED_TC_HOANTHANH': final_submission
})

filename = 'submission_final_stacking.csv'
submission_df.to_csv(filename, index=False)
print(f"\n🎉 Đã xuất file thành công: {filename}")
print(submission_df.head())


--- ✂️ 3. SPLITTING TRAIN/VALID/TEST ---
📌 Train size: (90582, 24)
📌 Valid size: (15144, 24)
📌 Test size:  (16502, 24)
📌 Categorical Features: ['PTXT', 'TOHOP_XT']

--- 🚀 4. ACTIVATING STACKING ENSEMBLE ---
   -> Training XGBoost...
   -> Training LightGBM...
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[266]	valid_0's rmse: 0.21248
   -> Training CatBoost...

--- 🔄 Layer 2: Blending & Prediction ---
📊 Trọng số Stacking: XGB: 0.22 | LGB: 0.29 | CAT: 0.49

>>> ✅ RMSE Stacking (Valid - HK2 23/24): 3.7427

🎉 Đã xuất file thành công: submission_final_stacking.csv
        MA_SO_SV  PRED_TC_HOANTHANH
2   00003e092652          14.947369
3   00027b0dec4c          17.174156
10  000e15519006          17.446657
13  000ea6e12003          15.980199
16  00109b845a3d           5.040422


In [7]:
# ...existing code...
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

# Đảm bảo final_credits_valid và y_valid_credits đã sẵn sàng
final_credits_valid = post_process_credits(final_rate_valid, valid_dangky)

rmse_val = np.sqrt(mean_squared_error(y_valid_credits, final_credits_valid))
r2_val = r2_score(y_valid_credits, final_credits_valid)

# MAPE (Chỉ tính trên những dòng có đăng ký tín chỉ > 0)
mask = y_valid_credits > 0
mape_val = mean_absolute_percentage_error(
    y_valid_credits[mask],
    final_credits_valid[mask]
)

print(f"--- 📊 KẾT QUẢ KIỂM TRA ---")
print(f"✅ RMSE : {rmse_val:.4f} (tín chỉ)")
print(f"✅ R^2  : {r2_val:.4f}")
print(f"✅ MAPE : {mape_val:.2%}")
# ...existing code...

--- 📊 KẾT QUẢ KIỂM TRA ---
✅ RMSE : 3.7427 (tín chỉ)
✅ R^2  : 0.7158
✅ MAPE : 29.43%
